In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code 
    true_positives = K.sum(y_true*y_pred)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def createModel(convFilters1, convFilters2, convFilters3, convFilters4, numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate):
    model = keras.Sequential()
        
    #convPool1
    model.add(keras.layers.Conv2D(convFilters1,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool2
    model.add(keras.layers.Conv2D(convFilters2,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #convPool3
    model.add(keras.layers.Conv2D(convFilters3,(3,3), activation='relu',padding='valid'))
    model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
    #finalConv
    model.add(keras.layers.Conv2D(convFilters4,(3,3), activation='relu',padding='valid'))
    
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='softmax'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
            model.add(keras.layers.Dropout(dropoutRate))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer="adam",loss=f1_loss, metrics=f1_score)
    return model

In [5]:
def createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                            n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                            dropoutRates,adamLearningRates,L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_convFilters1'] = n_convFilters1
    modelParameters['n_convFilters2'] = n_convFilters2
    modelParameters['n_convFilters3'] = n_convFilters3
    modelParameters['n_convFilters4'] = n_convFilters4
    modelParameters['n_FCLayers'] = n_FCLayers
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['dropoutRate'] = dropoutRates
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [6]:
def createRangeFromMidpoint(midpoint,range,mandatoryMinimum=1):
    possibleMin = int(midpoint-(range/2))
    possibleMin = max([mandatoryMinimum,possibleMin])
    possibleMax = int(midpoint+(range/2))
    possibleRange = np.arange(possibleMin,possibleMax)
    return possibleRange

In [7]:
def f1_loss(y_true,y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    f1 = (2 * precision * recall) / (precision + recall + K.epsilon())
    return 1 - f1

In [8]:
def generateDropoutRate(minVal=0,maxVal=1):
    dropoutRate = np.random.random() * (maxVal - minVal) + minVal
    dropoutRate = np.max([dropoutRate,0])
    dropoutRate = np.min([dropoutRate,0.999])
    return dropoutRate

In [9]:
def generateAdamLearningRate(minVal=1e-4,maxVal=1e-2):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    learningRatePower = np.random.random() * (maxVal - minVal) + minVal
    learningRate = np.power(10,learningRatePower)
    return learningRate

In [10]:
def generateL2(minVal=1e-2,maxVal=1e3):
    minVal = np.log10(minVal)
    maxVal = np.log10(maxVal)
    l2Power = np.random.random() * (maxVal - minVal) + minVal
    l2 = np.power(10,l2Power)
    return l2

In [11]:
def calculateCriticalPoints(top5ParamList):
    meanValue = np.mean(top5ParamList)
    stdValue = np.std(top5ParamList)
    lowPoint = meanValue - (3*stdValue)
    highPoint = meanValue + (3*stdValue)
    return (lowPoint,highPoint)

In [12]:
def calculateLogisticCriticalPoints(top5ParamList):
    top5Log10ParamList = np.log10(top5ParamList)
    log10criticalPointTuple = calculateCriticalPoints(top5Log10ParamList)
    criticalPointTuple = (np.power(10,log10criticalPointTuple[0]),np.power(10,log10criticalPointTuple[1]))
    return criticalPointTuple

In [13]:
def main():

    possibleConvFilters1 = createRangeFromMidpoint(32,32)
    possibleConvFilters2 = createRangeFromMidpoint(32,32)
    possibleConvFilters3 = createRangeFromMidpoint(32,32)
    possibleConvFilters4 = createRangeFromMidpoint(32,32)

    possibleNumberOfFCLayers = createRangeFromMidpoint(16,16)
    possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(16,16)

    possibleNumberOfEpochs = createRangeFromMidpoint(20,20)
    dropoutCriticalPoints = (0,1)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3)    
    
    trial = 0
    
    train = loadData("training")
    dev,test = loadData("testing",.5)

    choose = np.random.choice

    n_convFilters1 = []
    n_convFilters2 = []
    n_convFilters3 = []
    n_convFilters4 = []
    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    n_Epochs = []
    dropoutRates = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []
    
    while trial < 50:
        convFilters1 = choose(possibleConvFilters1)
        convFilters2 = choose(possibleConvFilters2)
        convFilters3 = choose(possibleConvFilters3)
        convFilters4 = choose(possibleConvFilters4)

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        dropoutRate = generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
        adamLearningRate = generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        

        model = createModel(convFilters1, convFilters2, convFilters3, convFilters4,
                            numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate)
    
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            n_convFilters1.append(convFilters1)
            n_convFilters2.append(convFilters2)
            n_convFilters3.append(convFilters3)
            n_convFilters4.append(convFilters4)
            
            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            dropoutRates.append(dropoutRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createModelParametersDF([convFilters1],[convFilters2],[convFilters3],[convFilters4],
                                                  [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
        
            
        if (trial % 10 == 9): 
            modelParametersDF = createModelParametersDF(n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,
                                                    n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                    dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleConvFilters1 = createRangeFromMidpoint(np.mean(top5['n_convFilters1']),6*np.std(top5['n_convFilters1']))
            possibleConvFilters2 = createRangeFromMidpoint(np.mean(top5['n_convFilters2']),6*np.std(top5['n_convFilters2']))
            possibleConvFilters3 = createRangeFromMidpoint(np.mean(top5['n_convFilters3']),6*np.std(top5['n_convFilters3']))
            possibleConvFilters4 = createRangeFromMidpoint(np.mean(top5['n_convFilters4']),6*np.std(top5['n_convFilters4']))
        
            possibleNumberOfFCLayers = createRangeFromMidpoint(np.mean(top5['n_FCLayers']),6*np.std(top5['n_FCLayers']))
            possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(np.mean(top5['n_NeuronsPerFCLayers']),6*np.std(top5['n_NeuronsPerFCLayers']))
        
            possibleNumberOfEpochs = createRangeFromMidpoint(np.mean(top5['n_Epochs']),6*np.std(top5['n_Epochs']))
            dropoutCriticalPoints = calculateCriticalPoints(top5['dropoutRate'])
            adamLearningRateCriticalPoints = calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = calculateLogisticCriticalPoints(top5['L2Rates'])

            n_convFilters1 = []
            n_convFilters2 = []
            n_convFilters3 = []
            n_convFilters4 = []
            n_FCLayers = []
            n_NeuronsPerFCLayers = []
            n_Epochs = []
            dropoutRates = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []
            
    

In [ ]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-07-05 22:38:15.513864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-05 22:38:15.514011: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-07-05 22:38:15.971493: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 9s 113ms/step - loss: 0.3511 - f1_score: 0.6490
devScore


2024-07-05 22:44:53.001856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-07-05 22:44:53.002010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 73ms/step - loss: 0.3657 - f1_score: 0.6277
concluding trial  0

trainScore
75/75 [==============================] - 10s 132ms/step - loss: 0.4053 - f1_score: 0.6487
devScore
10/10 [==============================] - 1s 90ms/step - loss: 0.4235 - f1_score: 0.6244
concluding trial  1

trainScore
75/75 [==============================] - 9s 117ms/step - loss: 0.3545 - f1_score: 0.6497
devScore
10/10 [==============================] - 1s 78ms/step - loss: 0.3703 - f1_score: 0.6384
concluding trial  2

trainScore
75/75 [==============================] - 8s 103ms/step - loss: 0.3504 - f1_score: 0.6496
devScore
10/10 [==============================] - 1s 64ms/step - loss: 0.3743 - f1_score: 0.6196
concluding trial  3

trainScore
75/75 [==============================] - 9s 109ms/step - loss: 63.0200 - f1_score: 0.6498
devScore
10/10 [==============================] - 1s 70ms/step - loss: 63.0374 - f1_score: 0.6320
concluding trial  4

trainScore
75/75

...

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
9,40,22,19,53,16,9,12,0.000000,0.005447,2.376538e-11,0.649810,0.638305
1,32,28,31,25,19,34,36,0.000000,0.000002,6.282269e+01,0.649682,0.634216
4,4,38,23,23,16,16,39,0.778292,0.000034,3.964611e+04,0.649381,0.642278
0,27,15,23,41,29,22,31,0.000000,0.015335,5.382469e-12,0.649201,0.629162
8,40,21,30,38,1,19,48,0.999000,0.000026,1.292312e+11,0.649171,0.631313
2,35,7,27,14,9,6,34,0.000000,0.000002,1.803099e+04,0.649163,0.641474
6,37,19,29,56,13,19,47,0.999000,0.131989,6.299598e-09,0.648273,0.641446
7,12,7,41,18,14,19,20,0.999000,0.000004,2.361828e+09,0.648112,0.638210
5,9,23,16,38,15,13,19,0.000000,0.000247,9.624961e-03,0.647920,0.650294
3,38,15,35,49,20,13,22,0.883772,0.002450,6.453646e-12,0.647737,0.628222



trainScore
75/75 [==============================] - 6s 74ms/step - loss: 0.3509 - f1_score: 0.6484
devScore
10/10 [==============================] - 1s 38ms/step - loss: 0.3684 - f1_score: 0.6454
concluding trial  29

trainScore
75/75 [==============================] - 13s 163ms/step - loss: 0.3509 - f1_score: 0.6481
devScore
10/10 [==============================] - 2s 119ms/step - loss: 0.3672 - f1_score: 0.6323
concluding trial  30

trainScore
75/75 [==============================] - 11s 142ms/step - loss: nan - f1_score: nan
devScore
10/10 [==============================] - 1s 97ms/step - loss: nan - f1_score: nan
concluding trial  31

trainScore
75/75 [==============================] - 7s 87ms/step - loss: 15788.4033 - f1_score: 0.6492
devScore
10/10 [==============================] - 1s 51ms/step - loss: 15788.4102 - f1_score: 0.6283
concluding trial  32

trainScore
75/75 [==============================] - 8s 97ms/step - loss: 118709760735313920.0000 - f1_score: 0.6479
devScore
1

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
8,19,19,33,41,34,4,4,0.000000,3.320229e+00,3.526939e-16,0.649991,0.639662
3,7,44,24,45,32,19,67,0.999000,9.862183e-09,4.573323e+08,0.649197,0.628325
0,12,12,20,20,19,38,50,0.114903,4.613078e-05,9.481873e-18,0.648401,0.645392
7,29,38,13,45,6,37,51,0.000000,1.950063e+00,9.562645e-20,0.648221,0.628137
1,42,38,30,11,15,27,22,0.675841,1.997825e+00,2.563984e-17,0.648131,0.632253
4,9,45,18,13,34,28,2,0.000000,2.347420e-02,5.978695e+14,0.647913,0.618049
5,19,29,28,10,17,22,18,0.000000,7.913826e-09,2.368076e-01,0.647210,0.633250
2,47,8,21,13,39,1,43,0.000000,1.326986e-04,2.272127e+21,NaN,NaN
6,8,45,22,6,26,41,14,0.545314,1.591190e-05,4.238175e+19,NaN,NaN
9,25,16,33,39,26,20,41,0.551967,5.908406e-05,1.077658e+21,NaN,NaN



trainScore
75/75 [==============================] - 9s 116ms/step - loss: 0.3500 - f1_score: 0.6501
devScore
10/10 [==============================] - 1s 77ms/step - loss: 0.3695 - f1_score: 0.6301
concluding trial  39

trainScore
75/75 [==============================] - 13s 165ms/step - loss: 0.3510 - f1_score: 0.6484
devScore
10/10 [==============================] - 2s 116ms/step - loss: 0.3671 - f1_score: 0.6324
concluding trial  40

trainScore
75/75 [==============================] - 12s 159ms/step - loss: 0.3493 - f1_score: 0.6506
devScore
10/10 [==============================] - 1s 104ms/step - loss: 0.3671 - f1_score: 0.6324
concluding trial  41

trainScore
75/75 [==============================] - 7s 79ms/step - loss: 0.3503 - f1_score: 0.6494
devScore
10/10 [==============================] - 1s 44ms/step - loss: 0.3671 - f1_score: 0.6376
concluding trial  42

trainScore
75/75 [==============================] - 9s 117ms/step - loss: 0.3508 - f1_score: 0.6494
devScore
10/10 [====

,n_convFilters1,n_convFilters2,n_convFilters3,n_convFilters4,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
2,34,52,20,28,23,33,22,0.976178,5.984436e-10,2.447510e-40,0.650640,0.632416
8,58,35,44,23,21,48,63,0.999000,5.379101e-10,7.944967e-37,0.650269,0.638469
0,24,55,2,36,35,56,74,0.000000,1.263281e-06,3.014951e-10,0.650060,0.630141
9,33,36,36,47,10,45,64,0.000000,2.238797e-03,2.733046e-01,0.649550,0.650921
4,23,32,29,19,23,31,81,0.999000,3.063655e-04,1.317664e-27,0.649412,0.630144
3,8,48,7,62,40,18,14,0.945597,1.174496e-01,1.055319e-28,0.649353,0.637648
5,40,9,10,54,33,12,24,0.999000,6.351074e-04,1.443176e-37,0.648727,0.628584
7,27,20,8,47,48,45,87,0.000000,2.129394e+03,1.231809e+08,0.648388,0.607238
1,45,36,34,9,27,51,60,0.000000,1.404624e-08,6.767609e-18,0.648365,0.632389
6,35,56,38,11,1,57,40,0.999000,2.978519e+03,6.215754e+02,0.647012,0.624598
